# Decision Analysis Project 1

## Installs and Imports

In [231]:
#Installs go here

In [232]:
#Imports go here
import warnings
import numpy as np
import pandas as pd

## Utilities

In [233]:
def calculate_value(value, q, p):
    if value > p:
        return 1.0
    elif value<=q:
        return 0
    return (value-q)/(p-q)

class Criteria:
    '''
    Criteria class for PROMETHEE and ELECTRE implementations.
    '''
    def __init__(self, name: str, criteria_type: str, w: float, q: float, p:float, v:float = float("inf")):
        '''
        Initialize instance of Criteria.
        Parameters:
            - name (str): Criteria name, e.g. (cost, completion time, rating, etc.)
            - criteria_type (str): Criteria type - 'gain' (higher values preffered) or 'cost' (lower values preffered)
            - w (float): Criteria weight
            - q (float): Indifference threshold
            - p (float): Perference threshold
            - v (float, optional): Veto threshold, default infinite -> used only for ELECTRE
        '''
        self.name = name
        assert criteria_type.lower() == "gain" or criteria_type.lower() == "cost", "Invalid criteria type"
        assert q<=p and p<=v and p>=0 and q>=0 and w>=0, "Invalid values, expected: q<p<v and q,p,w >=0"
        assert not np.isposinf(w) and not np.isneginf(w), "Invalid value, weight cannot be infinite"
        self.criteria_type = criteria_type.lower()
        self.w = w
        self.q = q
        self.p = p
        self.v = v

    def __str__(self):
        '''
        Print information about Criteria.
        '''
        return f"Criteria: {self.name} \n type: {self.criteria_type} \n w: {self.w} \n q: {self.q} \n p: {self.p} \n v: {self.v} \n"

## PROMETHEE

In [234]:
class PROMETHEE:
    def __init__(self, name: str = "MCDA Problem", criteria: list = [], alternatives = None, initialize_from_input: bool = False, initialize_from_file: bool = False, criteria_path = "", alternatives_path = "", version: int = 2):
        if alternatives is not None and not isinstance(alternatives, pd.DataFrame):
            raise TypeError("Alternatives must be a pandas DataFrame.")

        if not initialize_from_input and not initialize_from_file and len(criteria) == 0:
            warnings.warn("Criteria empty but idata from file or input is not read >:(", UserWarning)

        self.name = name
        if version != 1 and version != 2:
            raise ValueError("Only supported versions are 1 and 2")
        self.version = version

        if initialize_from_file:
            self.initalize_data_from_file(criteria_path, alternatives_path, criteria)

        elif initialize_from_input:
            self.initalize_data_from_input()

        else:
            self.criteria = criteria
            self.alternatives = alternatives

    def initalize_data_from_input(self):
        print("\n-----------------Initalize Criteria----------------\n")
        num_criteria = int(input("Num criteria = "))
        criterias = []
        for i in range(num_criteria):
            name = str(input("Name: "))
            cr_type = str(input("Type: "))
            w = int(input("Weight: "))
            q = int(input("Indifference Threshold: "))
            p = int(input("Preference Threshold: "))
            criterias.append(Criteria(name, cr_type, w, q, p))
        self.criteria = criterias

        print("\n------------------Initalize Alternatives--------------\n")
        num_alternatives = int(input("Num alternatives = "))
        alternatives = {criterias[i].name: [] for i in range(len(criterias))}
        alternatives["name"] = []
        for i in range(num_alternatives):
            name = str(input("ALternative Name: "))
            alternatives["name"].append(name)
            for ii in range(num_criteria):
                value = float(input(f"{criterias[ii].name} value: "))
                alternatives[criterias[ii].name].append(value)
        alternatives = pd.DataFrame.from_dict(alternatives)
        self.alternatives = alternatives

    def initalize_data_from_file(self, criteria_path, alternatives_path, criteria):
        if criteria_path == "":
            self.criteria = criteria
        else:
            self.criteria = pd.read_csv(criteria_path)
        self.alternatives = pd.read_csv(alternatives_path)

    def __str__(self):
        promethee_text = f"PROMETHEE {self.name}, version: {self.version}"
        for criterion in self.criteria:
            cr_printed = criterion.__str__()
            promethee_text += cr_printed
        promethee_text += "\n-----------------------Alternatives-----------------------\n"
        promethee_text += self.alternatives.__str__()
        return promethee_text

    def display(self):
        promethee_text = f"PROMETHEE {self.name}, version: {self.version}"
        for criterion in self.criteria:
            cr_printed = criterion.__str__()
            promethee_text += cr_printed
        promethee_text += "\n-----------------------Alternatives-----------------------\n"
        print(promethee_text)
        return self.alternatives.head(len(self.alternatives))

    def get_marginal_preference_index(self):
        marginal_index = {name: None for name in self.alternatives["name"]}
        copied_alts = self.alternatives.drop(columns=["name"])
        for i, row in copied_alts.iterrows():
            diff = copied_alts.sub(row)
            for crt in self.criteria:
                if crt.criteria_type == "gain":
                    diff[crt.name] *= -1
                diff[crt.name] = diff[crt.name].apply(calculate_value, args=(crt.q, crt.p))
            diff["name"] = self.alternatives["name"]
            marginal_index[self.alternatives["name"][i]]=diff
        return marginal_index

    def get_comprehensive_preference_index(self, marginal_index):
        comprehensive_preference_index = {name:[] for name in self.alternatives["name"]}
        weights = np.array([crit.w for crit in self.criteria])
        for alt_name in self.alternatives["name"]:
            temp_marginal_index = marginal_index[alt_name].drop(columns=["name"])
            temp_marginal_index = temp_marginal_index.values
            weighted_index = temp_marginal_index * weights
            result_sum = np.sum(weighted_index, axis=1)
            weight_sum = np.sum(weights)
            comprehensive_index_row = result_sum/weight_sum
            for i, val in enumerate(comprehensive_index_row):
                comprehensive_preference_index[self.alternatives["name"][i]].append(val)
        comprehensive_preference_index = pd.DataFrame.from_dict(comprehensive_preference_index)
        comprehensive_preference_index["name"] = self.alternatives["name"]
        return comprehensive_preference_index

    def get_positive_flow(self, comprehensive_preference_index):
        positive_flow = pd.DataFrame(comprehensive_preference_index.sum(axis=1), columns = ["Positive Flow"])
        positive_flow["name"] = comprehensive_preference_index["name"]
        return positive_flow

    def get_negative_flow(self, comprehensive_preference_index):
        negative_flow = pd.DataFrame(comprehensive_preference_index.sum(axis=0), columns = ["Negative Flow"])
        negative_flow.reset_index(inplace=True)
        negative_flow.rename(columns={'index': 'name'}, inplace=True)
        negative_flow = negative_flow[negative_flow['name'] != 'name']
        return negative_flow

    def get_complete_flow(self, positive_flow, negative_flow):
        names = positive_flow["name"]
        complete_flow = positive_flow.drop(columns=["name"]).values - negative_flow.drop(columns=["name"]).values
        return pd.DataFrame({'name': names, 'Complete Flow': complete_flow.flatten()})

    def solve(self):
        marginal_preference_index = self.get_marginal_preference_index()
        comprehensive_preference_index = self.get_comprehensive_preference_index(marginal_preference_index)
        positive_flow = self.get_positive_flow(comprehensive_preference_index)
        negative_flow = self.get_negative_flow(comprehensive_preference_index)
        if self.version == 1:
            #TODO: create ranking
            pass
        else:
            complete_flow = self.get_complete_flow(positive_flow, negative_flow)
            #TODO: fill by completing the version II algorithm
        return complete_flow

In [235]:
criteria = [
    Criteria(name = "price", criteria_type = "cost", w = 2.0, q = 3.0, p=9.0),
    Criteria(name = "critic_score", criteria_type = "gain", w = 1.0, q = 0.0, p=1.0),
    Criteria(name = "user_score", criteria_type = "gain", w = 5.0, q = 1.0, p=2.0),
    Criteria(name = "length", criteria_type = "gain", w = 6.0, q = 5.0, p=20.0),
    Criteria(name = "genres", criteria_type = "gain", w = 2.5, q = 9.0, p=15.0),
    Criteria(name = "num_of_achievements", criteria_type = "gain", w = 8.0, q = 3.3, p=92.0),
]

In [236]:
promete = PROMETHEE(initialize_from_file=True, criteria = criteria, alternatives_path = "/content/data.csv")

In [237]:
print(promete)

PROMETHEE MCDA Problem, version: 2Criteria: price 
 type: cost 
 w: 2.0 
 q: 3.0 
 p: 9.0 
 v: inf 
Criteria: critic_score 
 type: gain 
 w: 1.0 
 q: 0.0 
 p: 1.0 
 v: inf 
Criteria: user_score 
 type: gain 
 w: 5.0 
 q: 1.0 
 p: 2.0 
 v: inf 
Criteria: length 
 type: gain 
 w: 6.0 
 q: 5.0 
 p: 20.0 
 v: inf 
Criteria: genres 
 type: gain 
 w: 2.5 
 q: 9.0 
 p: 15.0 
 v: inf 
Criteria: num_of_achievements 
 type: gain 
 w: 8.0 
 q: 3.3 
 p: 92.0 
 v: inf 

-----------------------Alternatives-----------------------
                             name  price  critic_score  user_score  length  \
0          Dark Souls: Remastered    150            84          83      44   
1                  Dark Souls III    200            89          90      49   
2                        Terraria     46            81          81     102   
3                 Baldur's Gate 3    250            96          89     107   
4                  Dave the Diver     92            90          83      32   
5          

In [238]:
promete.display()

PROMETHEE MCDA Problem, version: 2Criteria: price 
 type: cost 
 w: 2.0 
 q: 3.0 
 p: 9.0 
 v: inf 
Criteria: critic_score 
 type: gain 
 w: 1.0 
 q: 0.0 
 p: 1.0 
 v: inf 
Criteria: user_score 
 type: gain 
 w: 5.0 
 q: 1.0 
 p: 2.0 
 v: inf 
Criteria: length 
 type: gain 
 w: 6.0 
 q: 5.0 
 p: 20.0 
 v: inf 
Criteria: genres 
 type: gain 
 w: 2.5 
 q: 9.0 
 p: 15.0 
 v: inf 
Criteria: num_of_achievements 
 type: gain 
 w: 8.0 
 q: 3.3 
 p: 92.0 
 v: inf 

-----------------------Alternatives-----------------------



,name,price,critic_score,user_score,length,genres,num_of_achievements
0,Dark Souls: Remastered,150,84,83,44,11,41
1,Dark Souls III,200,89,90,49,11,43
2,Terraria,46,81,81,102,9,115
3,Baldur's Gate 3,250,96,89,107,8,54
4,Dave the Diver,92,90,83,32,7,43
5,Rust,153,69,65,37,10,92
6,Hollow Knight,68,90,91,42,10,63
7,Portal 2,46,95,89,14,7,51
8,Vampire Survivors,20,86,83,25,9,204
9,Hades,115,93,88,49,9,49


In [239]:
result = promete.solve()

<ipython-input-234-e07fa97701cd>:105: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  positive_flow = pd.DataFrame(comprehensive_preference_index.sum(axis=1), columns = ["Positive Flow"])


In [240]:
result

,name,Complete Flow
0,Dark Souls: Remastered,-0.007436
1,Dark Souls III,3.864841
2,Terraria,10.389223
3,Baldur's Gate 3,7.867538
4,Dave the Diver,0.126066
5,Rust,-2.158452
6,Hollow Knight,7.986542
7,Portal 2,2.028061
8,Vampire Survivors,9.036575
9,Hades,5.304227


## SRF

## ELECTRE

## Use